# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [99]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [100]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\marce\Desktop\New folder


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [101]:
filename = 'base_de_dados.xlsx'

In [102]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificacao
0,quer fazer sua mulher feliz? come ela e da um ...,2
1,"@kimkataguiri que merda estão inventando, kim ...",0
2,alguém tem o link de spotify hackeado pra iphone?,0
3,@adr1nztte @charlidamelio mn pelo menos eu ain...,0
4,favela venceu e meu iphone tá vindo hj😍😘,0


In [103]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificacao
0,meu patrão quer que eu pego o iphone 12 pró ma...,0
1,comprei um m62 e vendi meu iphone me sinto com...,0
2,@omardodeboche @marcosrojerio2 @isabeledacpi e...,0
3,"alguém que tenha o iphone 7 plus, sabe me dize...",0
4,se algum amg gostasse de mim de vdd me dava um...,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o Iphone e consideramos tweets que demonstrassem a opinião popular referente ao iphone como sendo os tweets mais relevantes. Especificamente, consideramos a opinião popular quanto à características do próprio iphone, como câmera, bateria (elementos do hardware) e também caracteísticas mais subjetivas como o preço. Tweets que se distanciassem desses critérios de relevância foram avaliados como menos relevantes (evidentemente).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [104]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços -- extraída da aula 7, melhorar para removerr toda ponutacao e emojis
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed.lower()



opiniao = train.loc[(train["Classificacao"] == 3) | (train["Classificacao"] == 2),:]
nao_opiniao = train.loc[(train["Classificacao"] == 0) | (train["Classificacao"] == 1),:]


lista_tweets_limpa = [cleanup(tweet) for tweet in train["Treinamento"].to_list()]

texto_opiniao_limpo = cleanup(" ".join(opiniao["Treinamento"].to_list()))
texto_nao_opiniao_limpo = cleanup(" ".join(nao_opiniao["Treinamento"].to_list()))


lista_palavras_opiniao = texto_opiniao_limpo.split(" ")
lista_palavras_nao_opiniao = texto_nao_opiniao_limpo.split(" ")


palavras_opiniao = " ".join(lista_palavras_opiniao)
palavras_nao_opiniao = " ".join(lista_palavras_nao_opiniao)



lista_palavras_relativa_opiniao = pd.Series(lista_palavras_opiniao).value_counts(normalize=True)
lista_palavras_relativa_nao_opiniao = pd.Series(lista_palavras_nao_opiniao).value_counts(normalize=True)


#preciso remover stopwords e @ das primeiras listas

prior_op = len(lista_palavras_opiniao)/(len(lista_palavras_opiniao)+len(lista_palavras_nao_opiniao))
prior_nop = len(lista_palavras_nao_opiniao)/(len(lista_palavras_opiniao)+len(lista_palavras_nao_opiniao))

def classify(tweet):
    palavras_tweet = tweet.split(" ")
    p_opiniao = []
    pn_opiniao = []
    for palavra in palavras_tweet:
        if palavra in lista_palavras_relativa_opiniao:
            p_opiniao.append(lista_palavras_relativa_opiniao[palavra]+1)
        else:
            p_opiniao.append(1)
        if palavra in lista_palavras_relativa_nao_opiniao:
            pn_opiniao.append(lista_palavras_relativa_nao_opiniao[palavra]+1)
        else:
            pn_opiniao.append(1)
    p_opiniao_resultado = np.prod(p_opiniao)*prior_op
    pn_opiniao_resultado = np.prod(pn_opiniao)*prior_nop
    if p_opiniao_resultado >= pn_opiniao_resultado:
        return "Opinião"
    else:
        return "Isento"

lista_tags = []

for tweet in lista_tweets_limpa:
    tag = classify(tweet)
    lista_tags.append(tag)

train["Tag"] = lista_tags

print(prior_nop)
print(prior_op)
print(len(train.loc[train["Tag"]=="Opinião",:]))
print(len(train.loc[train["Tag"]=="Isento",:]))



0.760941212539281
0.23905878746071893
0
750


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**